In [60]:
import pandas as pd
import numpy as np
import pendulum
from dotenv import load_dotenv
import os

In [61]:
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest 
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.enums import AssetStatus
from alpaca.trading.enums import AssetClass
from alpaca.data.enums import Adjustment
from alpaca.data.timeframe import TimeFrameUnit

In [62]:
# Environment Variables
load_dotenv()

api_key = os.getenv("ALPACA_API_KEY")
secret_key = os.getenv("ALPACA_SECRET_KEY")

In [63]:
secret_key

'vnXiLRIPQPvD5gQB3oVJ3p6mlxkYB97BFprjfXt0'

In [64]:
# clients
data_client = StockHistoricalDataClient(api_key,secret_key)
trading_client = TradingClient(api_key,secret_key)

In [65]:
# Parameters
end = pendulum.datetime(2024,7,4)
start = end.subtract(days=2)

In [66]:
# Get all current assets
asset_request = GetAssetsRequest(
    status = AssetStatus.ACTIVE,
    asset_class = AssetClass.US_EQUITY,
)

all_assets = trading_client.get_all_assets(asset_request)

all_assets = [vars(asset) for asset in all_assets] # Convert objects to dicts

In [67]:
df = pd.DataFrame(data=all_assets)

df = df[df['tradable'] & df['fractionable'] & df['shortable']]

df.head()

,id,asset_class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable,min_order_size,min_trade_increment,price_increment,maintenance_margin_requirement,attributes
475,4e1dc051-9fa4-479d-982e-35a1c60efadc,AssetClass.US_EQUITY,AssetExchange.ARCA,IPAC,iShares Core MSCI Pacific ETF,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"
477,7a275005-aaaa-42c4-b6b2-f14a37376f67,AssetClass.US_EQUITY,AssetExchange.ARCA,SCHB,Schwab U.S. Broad Market ETF,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"
480,d8a180f7-e763-4d30-bc36-660b80830acc,AssetClass.US_EQUITY,AssetExchange.ARCA,VIG,Vanguard Dividend Appreciation ETF,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"
481,a0c65d4a-454a-4444-9c58-197e97235ed3,AssetClass.US_EQUITY,AssetExchange.NASDAQ,VIGI,Vanguard Whitehall Funds Vanguard Internationa...,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,"[fractional_eh_enabled, has_options]"
485,a02bae52-419a-4c0c-b6b5-086ef09d764b,AssetClass.US_EQUITY,AssetExchange.BATS,ESGV,Vanguard ESG U.S. Stock ETF,AssetStatus.ACTIVE,True,True,True,True,True,None,None,None,30.0,[fractional_eh_enabled]


In [68]:
symbols = df['symbol'].to_list()

In [69]:
# Get previous days market data
bars_request = StockBarsRequest(
    symbol_or_symbols=symbols,
    timeframe=TimeFrame(1,TimeFrameUnit.Day),
    start=start,
    end=end,
    adjustment=Adjustment.ALL,
)

raw_bars= data_client.get_stock_bars(bars_request)

In [70]:
bars = []
for symbol, data_list in raw_bars.data.items():
    for data in data_list:
        bars.append(vars(data))

df = pd.DataFrame(bars)

df

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AMP,2024-07-02 04:00:00+00:00,426.06,434.62,426.060,433.87,404793.0,12935.0,432.439958
1,AMP,2024-07-03 04:00:00+00:00,435.13,438.34,431.865,434.00,182915.0,6769.0,433.777125
2,AMTB,2024-07-02 04:00:00+00:00,22.90,23.00,22.680,22.92,97480.0,2061.0,22.883858
3,AMTB,2024-07-03 04:00:00+00:00,22.84,22.93,22.450,22.50,89348.0,1707.0,22.646498
4,ASTE,2024-07-02 04:00:00+00:00,28.93,29.43,28.830,29.16,125989.0,2821.0,29.163862
...,...,...,...,...,...,...,...,...,...
7371,XRX,2024-07-03 04:00:00+00:00,11.78,11.83,11.585,11.61,1015314.0,10567.0,11.698648
7372,YY,2024-07-02 04:00:00+00:00,30.34,31.09,30.215,31.01,352042.0,5596.0,30.872281
7373,YY,2024-07-03 04:00:00+00:00,31.35,32.60,31.250,31.90,412610.0,5695.0,32.079111
7374,ZH,2024-07-02 04:00:00+00:00,2.72,2.73,2.650,2.73,364635.0,2765.0,2.693926
